# Loading the train dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
pd.options.display.max_colwidth = 250

SyntaxError: invalid syntax (<ipython-input-1-c24fbe38da89>, line 8)

In [ ]:
#Loading the train dataset
train_df = pd.read_csv("../input/train-and-test/my_train_set.csv")

In [2]:
train_df.sample(5)

NameError: name 'train_df' is not defined

In [ ]:
train_df.shape

In [ ]:
def sub_num(text):
    #substitutinging numbers
    text = re.sub('[0-9]+', 'numbr', text)
    return text

In [ ]:
train_df['clean_text'] = train_df['clean_text'].apply(str).apply(lambda x: sub_num(x))

In [ ]:
#Removing all stopwords
from nltk.corpus import stopwords
my_stopwords = set(stopwords.words('english'))

def clean_stopwords(text):
    item = [item for item in text.split(' ') if item not in my_stopwords]
    return ' '.join(item)

train_df['clean_text'] = train_df['clean_text'].apply(str).apply(lambda x: clean_stopwords(x))
train_df.sample(5)

In [ ]:
train_df.sample(5)

In [ ]:
train_df['review_len'].describe()

# **Bi-directional Long Short-Term Memory**



# Building Model

In [ ]:
import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Sequential
from keras.preprocessing import sequence
from collections import Counter
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


import decimal
import tensorflow as tf
import re
from keras.models import load_model


avg = 'weighted'
num_classes = 4
ROUND_PREC = 1
NUM_EPOCHS = 200  # Will use early stopping to determine num. of epochs
encoder = LabelEncoder()
plot_labels = []


In [ ]:
from keras.preprocessing.text import Tokenizer
top_words = 8000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['clean_text'])

X_train = tokenizer.texts_to_sequences(train_df['clean_text'])
#X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1 

print(vocab_size)

In [ ]:
print(train_df.Text[2])

In [ ]:
print(train_df.clean_text[2])

In [ ]:
print(X_train[2])

In [ ]:
from keras.preprocessing.sequence import pad_sequences
#X = sequence.pad_sequences(train_vector, maxlen=550)  # padding the training set
X = pad_sequences(X_train, padding='post', maxlen=500)
predictor_set = train_df['ratings']
Y = keras.utils.np_utils.to_categorical(predictor_set)


print('\n\t\t\t\t.............................Task 1 begins ................................')

print('\nChecking the different ratios of ratings in the predictor set: {} '.format(Counter(predictor_set)))

#Checking the different labels
encoder.fit(predictor_set)
labels = []
for i in range(0, 4):
    labels.append(encoder.inverse_transform([i])[0])
print('\nThe different lables are: ', labels)

#Splitting the data into training and validation set( 70 : 30)
print('\nSplitting our dataset into training and validatin set...')
train_x, x_val, train_y, y_val = train_test_split(X, Y, stratify = Y, test_size = 0.3, random_state = 42)
print('\nshape of x_val:', x_val.shape)
print('shape of y_val:', y_val.shape)

print('\n\t\t\t\t..............................Task 1 compleeted ............................')


In [ ]:
print(train_x.shape[0])

# Initializing our TPU

In [ ]:
import tensorflow as tf
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
max_seq_len = 500
lstm_dim  = 100 
batch_size = 200   
dropout = 0.7   
emb_dim  = 100   
learning_rate = 0.0001 
num_classes = 4
loss = 'categorical_crossentropy'
metric = 'categorical_accuracy'
    

In [ ]:
from keras.layers import SpatialDropout1D
def build_model():#Training
    print('Training...')

    model = Sequential()
    
    #model.add(Embedding(vocab_size, emb_dim))
    model.add(Embedding(top_words, emb_dim, input_length=max_seq_len))
    
    #model.add(SpatialDropout1D(0.7))

    model.add(Bidirectional(LSTM(lstm_dim, dropout=dropout, recurrent_dropout=dropout)))
    #model.add(LSTM(lstm_dim, dropout=dropout, recurrent_dropout=dropout))

    model.add(Dense(num_classes))

    model.add(Activation('softmax'))

    return (model)

In [ ]:
with tpu_strategy.scope():
    model = build_model()
     
#metric = 'categorical_accuracy'eval_metrics = ['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy']
cur_optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=cur_optimizer, metrics=['categorical_accuracy']) #we will be monitoring the categorical accuracy

In [ ]:
model.summary()

In [ ]:
# Call backs for early stopping (https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min')

checkpoint_path = "weights_base.best.hdf5"  #Changed this
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=1)

history = model.fit(train_x, train_y, batch_size=batch_size, 
                        epochs=200, validation_data= (x_val, y_val), callbacks=[early_stop,checkpoint])


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

In [ ]:
history=history.history

In [ ]:
#Determing the different classes in our dataset
import itertools
num_classes = 4
label_dict = {'all_negs': 0, 'Movies':1, 'Clothing' : 2 , 'Fine_food' :3}

def diff_classes (num_classes):
    for i in range(0, num_classes):
        plot_labels.append(i)
    return(plot_labels)


plot_classes = diff_classes(num_classes)

def compute_confusion_matrix(cm, labels , title='plot confusion matrix', cmap=plt.cm.Blues):
        
        fig, ax = plt.subplots(figsize=(5,5))

        plt.imshow(cm, cmap=cmap )
        plt.title(title)
        tick_marks = np.arange(len(labels))
        plt.xticks(tick_marks, labels, rotation=45)
        plt.yticks(tick_marks, labels)

        fmt = 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.show();


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.metrics import f1_score
def model_evaluate(validation_data = ()):
    fs = []
    x_val, y_val = validation_data

    y_pred = np.asarray(model.predict(x_val)) # getting for one epoch
    y_true = y_val

    y_pred_final, y_true_final = ([] for _ in range(2))

    for true, pred in zip(y_true, y_pred):
        max_i_pred = (np.argsort(pred)[::-1])[0]  #then sorting the result per epoch and fininding the max_pred for that epoch
        max_pred_score = pred[max_i_pred]      #fininding the score for the prediction
        max_i_true = (np.argsort(true)[::-1])[0]
        max_true_score = true[max_i_true]

        y_pred_final.append(max_i_pred)

        y_true_final.append(max_i_true)


    fs = f1_score(y_true_final, y_pred_final, average=None)

    #defining and plotting our confusion matrix

    print('The confusion matrix: ')
    cnf_matrix = confusion_matrix(y_true_final, y_pred_final)
    plt.figure()

    compute_confusion_matrix(cnf_matrix, labels=labels,
                          title='plot_Confusion_matrix');


    #Gettign the classification report
    print('The classification report: ')
    target_names = ['class_' + i for i in [str(i) for i in list(plot_classes)]]
    print(classification_report(y_true_final, y_pred_final, target_names=target_names));


    print('f1_scores per classs are:\n f1_score = {}'.format(fs))

In [ ]:
model_evaluate(validation_data = (x_val, y_val))

In [ ]:
best_epoch_metrics = []

# Determine if best loss is last epoch or the one before it
#epoch number is 0based
best_epoch_num = (len(history['val_loss'])-2)  if (history['val_loss'][len(history['val_loss'])-1] > history['val_loss'][len(history['val_loss'])-2]) else (len(history['val_loss'])-1) 
best_epoch_metrics.append(str(best_epoch_num))
print ('The best Epoch:', best_epoch_num )

# TESTING ON TEST_SET

In [ ]:
from keras.models import load_model
test_model = load_model('weights_base.best.hdf5')


In [ ]:
# Importing the test set initially saved
test_df = pd.read_csv("../input/train-and-test/my_test_set.csv")

In [ ]:
test_df.head()

In [ ]:
test_df['clean_text'] = test_df['clean_text'].apply(str).apply(lambda x: sub_num(x))

In [ ]:
test_df['clean_text'] = test_df['clean_text'].apply(str).apply(lambda x: clean_stopwords(x))
test_df.sample(5)

In [ ]:
test_df['review_len'].describe()

In [ ]:
tokenizer.fit_on_texts(test_df['clean_text'])

test_data = tokenizer.texts_to_sequences(test_df['clean_text'])

In [ ]:
#padding the test_features
test_features = pad_sequences(test_data, padding='post', maxlen=500)
test_labels = keras.utils.np_utils.to_categorical(test_df['ratings'])


In [ ]:
model_evaluate(validation_data = (test_features, test_labels));

In [ ]:
###################################################################################################